In [ ]:
import json
from tqdm.auto import tqdm

In [2]:
with open('../Data/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
documents[3]

{'intent': 'receive_payment',
 'question': 'i got to receive fucking compensations can i get some ehlp',
 'response': 'To process your compensation payment for the insurance claim, please adhere to the following steps:\n\n1. Access your account by visiting {{WEBSITE_URL}}.\n2. Locate the {{CLAIM_SECTION}} for your specific claim.\n3. Identify the claim that you wish to be compensated for.\n4. Opt for the {{RECEIVE_PAYMENT_OPTION}} that best suits your preference.\n5. Follow the provided instructions to finalize your preferred method of payment and validate the information.\n\nShould you face any complications during this process, do not hesitate to reach out to our customer support team for immediate assistance.',
 'category': 'CLAIMS',
 'id': 'da0592e6'}

In [4]:
prompt_template = """
You are emulating a real customer contacting an insurance company's support.
Based on the following FAQ record, generate 5 natural questions the customer
might ask to get the same information. 

Guidelines:
- Each question should be clear, concise, and sound like something a real customer would say.  
- Do not reuse exact wording from the FAQ record — rephrase naturally.  
- Ensure all questions are relevant to the intent.  
- Cover variations (short, direct, polite, urgent, explanatory).  
- Keep questions simple and customer-friendly.  

FAQ record:
intent: {intent}
question: {question}
response: {response}

Return only valid JSON in this format (no explanations, no code blocks):

{{
  "questions": [
    "question1",
    "question2",
    "question3",
    "question4",
    "question5"
  ]
}}
""".strip()

In [5]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    json_response = response.choices[0].message.content
    return json_response



In [ ]:
results = {}

for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions
    with open('my_dict.json', 'w') as json_file:
        json.dump(results, json_file)

  0%|          | 0/390 [00:00<?, ?it/s]

In [ ]:
# import json
# from concurrent.futures import ThreadPoolExecutor, as_completed
# from tqdm.auto import tqdm

# # worker function for parallel execution
# def process_doc(doc):
#     doc_id = doc["id"]
#     questions = generate_questions(doc)  # your existing function
#     return doc_id, questions

# results = {}

# # ThreadPoolExecutor with, say, 8 workers (tune this for your M1 Mac)
# with ThreadPoolExecutor(max_workers=8) as executor:
#     # submit all tasks
#     futures = {executor.submit(process_doc, doc): doc for doc in documents}
    
#     # iterate over results as they complete
#     for future in tqdm(as_completed(futures), total=len(futures)):
#         doc_id, questions = future.result()
#         results[doc_id] = questions
        
#         # write progressively so you don’t lose progress
#         with open("my_dict.json", "w", encoding="utf-8") as json_file:
#             json.dump(results, json_file, ensure_ascii=False, indent=2)

# print("✅ Parallel processing complete. Results saved to my_dict.json")
